In [1]:
import pandas as pd

In [6]:
lemma_result = pd.read_csv("../Clean/lemma_result.csv") #Dataset

In [7]:
print(lemma_result.head())

                                      cleaned_review  Label
0  one reviewers mention watch 1 oz episode hook ...      1
1  wonderful little production film technique una...      1
2  think wonderful way spend time hot summer week...      1
3  basically family little boy jake think zombie ...      0
4  petter mattei love time money visually stun fi...      1


In [8]:
print(lemma_result.shape)

(1000, 2)


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(lemma_result['cleaned_review'])

lemma_result['cleaned_review'][1]

'wonderful little production film technique unassuming old time bbc fashion give comfort sometimes discomforting sense realism entire piece actors extremely well choose michael sheen get polari voice pat truly see seamless edit guide reference williams diary entries well worth watch terrificly write perform piece masterful production one great master comedy life realism really come home little things fantasy guard rather use traditional dream techniques remain solid disappear play knowledge sense particularly scenes concern orton halliwell set particularly flat halliwell murals decorate every surface terribly well do'

In [17]:
print([X[1, tfidf.vocabulary_['discomforting']]])

[0.1542468096810376]


In [18]:
lemma_result.groupby('Label').size()

Label
0    499
1    501
dtype: int64

In [21]:
from sklearn.model_selection import train_test_split
X = lemma_result.cleaned_review
y = lemma_result.Label
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [22]:
print("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(X_train),
                                                                             (len(X_train[y_train == 0]) / (len(X_train)*1.))*100,
                                                                            (len(X_train[y_train == 1]) / (len(X_train)*1.))*100))

Train set has total 750 entries with 48.27% negative, 51.73% positive


In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout,Dropout_U,Dropout_W
from sklearn.model_selection import train_test_split
import re

In [35]:
max_fatures = 3000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(lemma_result['cleaned_review'].values)
X1 = tokenizer.texts_to_sequences(lemma_result['cleaned_review'].values)
X1 = pad_sequences(X1)

In [36]:
Y1 = pd.get_dummies(lemma_result['Label']).values
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,Y1, random_state = 42)
print(X1_train.shape,Y1_train.shape)
print(X1_test.shape,Y1_test.shape)

(750, 505) (750, 2)
(250, 505) (250, 2)


In [45]:
embed_dim = 150
lstm_out = 200

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X1.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 505, 150)          450000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 505, 150)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               280800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 731,202
Trainable params: 731,202
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
batch_size = 32
history = model.fit(X1_train, Y1_train, epochs = 10, batch_size=batch_size, verbose = 2)

Train on 750 samples
Epoch 1/10
750/750 - 24s - loss: 0.6910 - accuracy: 0.5093
Epoch 2/10
750/750 - 23s - loss: 0.5883 - accuracy: 0.7707
Epoch 3/10
750/750 - 23s - loss: 0.3347 - accuracy: 0.9133
Epoch 4/10
750/750 - 21s - loss: 0.1953 - accuracy: 0.9307
Epoch 5/10
750/750 - 21s - loss: 0.0943 - accuracy: 0.9800
Epoch 6/10
750/750 - 21s - loss: 0.0242 - accuracy: 0.9947
Epoch 7/10
750/750 - 21s - loss: 0.0103 - accuracy: 0.9987
Epoch 8/10
750/750 - 22s - loss: 0.0079 - accuracy: 0.9987
Epoch 9/10
750/750 - 20s - loss: 0.0139 - accuracy: 0.9973
Epoch 10/10
750/750 - 21s - loss: 0.0248 - accuracy: 0.9973


In [48]:
score,acc = model.evaluate(X1_test, Y1_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

250/250 - 1s - loss: 0.8859 - accuracy: 0.6800
score: 0.89
acc: 0.68
